In [ ]:
%reload_ext autoreload
%autoreload 2
%pylab inline

import os
import pickle
import random
from collections import namedtuple
from itertools import count

import gym
import numpy as np
import pycuber
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
import tqdm
from gym import spaces
from gym.utils import seeding
from tensorboardX import SummaryWriter
from torch.distributions import Categorical

import cube
import dqn
import solver

In [ ]:
path_weights = 'weights/dqn_mean_100_gamma_0.9_batch_512_episodes_100000.pkl'

policy = dqn.DQN(n_space=cube.N_SPACE, n_action=cube.N_ACTION)
policy.load_state_dict(pickle.load(open(path_weights, 'rb')))

test_cube = cube.CubeEnv(steps=5)

In [ ]:
def test_solver(steps, solver, n_iter, time_limit):
    success_rate = []
    actions_length = []
    solve_times = []

    for _ in range(n_iter):
        solving_cube = cube.get_shuffled_cube(steps=steps)
        start_time = time.time()
        is_done, actions, depth, value = \
            solver.solve(solving_cube, time_limit=time_limit)
        solve_time = time.time() - start_time
        
        success_rate.append(is_done)
        if is_done:
            actions_length.append(len(actions))
            solve_times.append(solve_time)
            
    return np.mean(success_rate), actions_length, solve_times

In [ ]:
mcts = solver.SimpleMCTSSolver(policy, tau=1.5)
greedy = solver.GreedySolver(policy)

In [ ]:
result = {
    'greedy': {},
    'naive': {}}

In [ ]:
for steps in range(1, 11):
    print(steps)
    
    success_rate, actions_length, solve_times = \
        test_solver(steps=steps, solver=greedy, n_iter=100, time_limit=10)
    result['greedy'][steps] = {
        'success_rate': success_rate,
        'actions_length': actions_length,
        'solve_times': solve_times}    
    print('greedy', success_rate)
    
    success_rate, actions_length, solve_times = \
        test_solver(steps=steps, solver=mcts, n_iter=100, time_limit=10)
    result['naive'][steps] = {
        'success_rate': success_rate,
        'actions_length': actions_length,
        'solve_times': solve_times}
    print('naive', success_rate)
    
    print('-' * 10)

In [ ]:
result = pickle.load(open('results/test.pkl', 'rb'))

In [ ]:
result